# Calculate Average Width & Height
In this notebook, we will calculate avergae dimensions of each category. After that, we will crop images based on that and see whether are we able to get precise sub-images or not.

In [1]:
import json

base_dir = r"U:\Study\Durham_AI_Course\CapstoneOne\data\\"

with open(base_dir + r"train.json","r") as file:
    train_data = json.load(file)
    
with open(base_dir + r"val.json","r") as file:
    val_data = json.load(file)
    
train_annotations = train_data['annotations']
train_images = train_data['images']
categories = train_data['categories']

val_annotations = val_data['annotations']
val_images = val_data['images']

In [2]:
category_mapping = {}

for category_item in categories:
    category_mapping[category_item['id']] = category_item['name']

train_id_to_path_mapping = {}

for image_item in train_images:
    train_id_to_path_mapping[image_item['id']] = image_item['file_name']
    
val_id_to_path_mapping = {}

for image_item in val_images:
    val_id_to_path_mapping[image_item['id']] = image_item['file_name']
    
for annotation in train_annotations:
    annotation['image_path'] = 'train\\train\\'+train_id_to_path_mapping[annotation['image_id']]
    annotation['cat'] = category_mapping[annotation['category_id']]
    annotation['bbox'] = list(map(int,annotation['bbox']))
    
for annotation in val_annotations:
    annotation['image_path'] = 'val\\val\\'+val_id_to_path_mapping[annotation['image_id']]
    annotation['cat'] = category_mapping[annotation['category_id']]
    annotation['bbox'] = list(map(int,annotation['bbox']))
    
print("%.1000s" % train_annotations)
print("%.1000s" % val_annotations)

[{'area': 2304645, 'iscrowd': 0, 'id': 1, 'image_id': 10, 'category_id': 4, 'bbox': [704, 620, 1401, 1645], 'image_path': 'train\\train\\10.jpg', 'cat': 'dresses'}, {'area': 55769, 'iscrowd': 0, 'id': 2, 'image_id': 1000, 'category_id': 1, 'bbox': [321, 332, 217, 257], 'image_path': 'train\\train\\1000.jpg', 'cat': 'tops'}, {'area': 17108, 'iscrowd': 0, 'id': 3, 'image_id': 10003, 'category_id': 2, 'bbox': [220, 758, 188, 91], 'image_path': 'train\\train\\10003.jpg', 'cat': 'trousers'}, {'area': 117909, 'iscrowd': 0, 'id': 4, 'image_id': 10003, 'category_id': 5, 'bbox': [150, 397, 297, 397], 'image_path': 'train\\train\\10003.jpg', 'cat': 'skirts'}, {'area': 105925, 'iscrowd': 0, 'id': 5, 'image_id': 10003, 'category_id': 1, 'bbox': [85, 207, 475, 223], 'image_path': 'train\\train\\10003.jpg', 'cat': 'tops'}, {'area': 3314505, 'iscrowd': 0, 'id': 6, 'image_id': 10005, 'category_id': 4, 'bbox': [1971, 2059, 1483, 2235], 'image_path': 'train\\train\\10005.jpg', 'cat': 'dresses'}, {'area'

In [3]:
len(train_annotations)

13317

In [4]:
len(val_annotations)

2458

In [5]:
# Combine Annotations
annotations = train_annotations + val_annotations
len(annotations)

15775

In [6]:
category_mapping

{1: 'tops', 2: 'trousers', 3: 'outerwear', 4: 'dresses', 5: 'skirts'}

## Collect category wise dimensions

Utility funcations

In [7]:
def get_cropped_image(img, bbox):
    start_x, start_y, width, height = bbox
    cropped_img = img[start_y:start_y+height, start_x:start_x+width]
    return cropped_img
      
def get_reshaped_image(img, new_shape=(224,224)):
    resized_image = cv2.resize(img, new_shape, interpolation = cv2.INTER_NEAREST) 
    return resized_image

def rescale_bbox(bbox, current_img_shape, new_img_shape=(224,224)):
    x_ratio = new_img_shape[0] / current_img_shape[0]
    y_ratio = new_img_shape[1] / current_img_shape[1]
    
    new_x = bbox[0] * x_ratio
    new_y = bbox[1] * y_ratio
    new_width = bbox[2] * x_ratio
    new_height = bbox[3] * y_ratio
    
    return new_x, new_y, new_width, new_height

Calculate Averages

In [8]:
import cv2

from tqdm import tqdm 

new_image_shape = (128,128)

dimensiosn = {
    1: {'width':[], 'height':[]},
    2: {'width':[], 'height':[]},
    3: {'width':[], 'height':[]},
    4: {'width':[], 'height':[]},
    5: {'width':[], 'height':[]},
}

# bbox -> x, y, width, height
for annot in tqdm(annotations):
    cat = annot['category_id']
    bbox = annot['bbox']
    
    img = cv2.imread(base_dir+annot['image_path'])
    bbox = rescale_bbox(bbox,img.shape,new_img_shape=new_image_shape)
    
    dimensiosn[cat]['width'].append(bbox[2])
    dimensiosn[cat]['height'].append(bbox[3])
    
for cat, items in dimensiosn.items():
    total = len(items['width'])
    width_avg = sum(items['width'])/total
    height_avg = sum(items['height'])/total
    
    print(f"Category: {cat}, Width Avg: {width_avg}, Height Avg: {height_avg}")

100%|████████████████████████████████████████████████████████████████████████████| 15775/15775 [57:09<00:00,  4.60it/s]


Category: 1, Width Avg: 32.077987395031144, Height Avg: 46.673156901322756
Category: 2, Width Avg: 22.002879686807113, Height Avg: 44.81315624124854
Category: 3, Width Avg: 41.15369807284264, Height Avg: 58.66411483191084
Category: 4, Width Avg: 40.08467534174697, Height Avg: 81.91231424485893
Category: 5, Width Avg: 39.87580947143186, Height Avg: 55.75588302093967
